

This tutorial will guide you through finding a notebook on the website, converting it to a ploomber pipeline and expose the pipeline to Airflow.

## Setting up environment
The following commands will install prerequisites including:
* [Ploomber](https://github.com/ploomber/ploomber): Ploomber is a framework to build collaborative and modular pipelines; it integrates with Jupyter but you can use it with any other editor.
* [Soorgeon](https://github.com/ploomber/soorgeon): Soorgeon converts monolithic Jupyter notebooks into maintainable Ploomber pipelines.
* [Soopervisor](https://github.com/ploomber/soopervisor): Soopervisor runs Ploomber pipelines for batch processing (large-scale training or batch serving) or online inference.
* [Airflow](https://airflow.apache.org/): Airflow is a platform to programmatically author, schedule and monitor workflows.

In [1]:
%%bash
pip -q install 'apache-airflow==2.3.3' \
 --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.3.3/constraints-3.7.txt"
pip -q install ploomber soorgeon soopervisor
pip -q install -r https://github.com/ploomber/ploomber/raw/master/requirements-colab.lock.txt

export AIRFLOW__CORE__LOAD_EXAMPLES=False
airflow db init
airflow users create \
    --username ploomber \
    --firstname Peter \
    --lastname Parker \
    --role Admin \
    --email spiderman@superhero.org \
    --password ploomber

DB: sqlite:////root/airflow/airflow.db
[2022-08-05 22:13:04,949] {db.py:1462} INFO - Creating tables
[2022-08-05 22:13:08,625] {manager.py:244} INFO - Inserted Role: Admin
[2022-08-05 22:13:08,636] {manager.py:244} INFO - Inserted Role: Public
[2022-08-05 22:13:08,638] {manager.py:817} WARNING - No user yet created, use flask fab command to do it.
[2022-08-05 22:13:09,789] {manager.py:508} INFO - Created Permission View: can delete on Connections
[2022-08-05 22:13:09,808] {manager.py:508} INFO - Created Permission View: can read on Connections
[2022-08-05 22:13:09,826] {manager.py:508} INFO - Created Permission View: can edit on Connections
[2022-08-05 22:13:09,851] {manager.py:508} INFO - Created Permission View: can create on Connections
[2022-08-05 22:13:09,907] {manager.py:508} INFO - Created Permission View: can read on DAGs
[2022-08-05 22:13:09,929] {manager.py:508} INFO - Created Permission View: can edit on DAGs
[2022-08-05 22:13:09,948] {manager.py:508} INFO - Created Permissi

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.0 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
sphinx 1.8.6 requires docutils<0.18,>=0.11, but you have docutils 0.19 which is incompatible.
spacy 3.4.1 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
pytest 3.6.4 requires pluggy<0.8,>=0.5, but you have pluggy 1.0.0 which is incompatible.
nbclient 0.6.6 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.
nbclient 0.6.6 requires traitlets>=5.2.2, but you have traitlets 5.1.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

## Preparing notebook and data

We selected a logistic regression example from the [Titanic - Machine Learning from Disaster competition](https://www.kaggle.com/c/titanic) on Kaggle. We will download the notebook from the author's Github repository [mnassrib/Titanic-logistic-regression-with-python](https://github.com/mnassrib/Titanic-logistic-regression-with-python).

The original notebook is available at: [Titanic: logistic regression with python | Kaggle](https://www.kaggle.com/code/mnassrib/titanic-logistic-regression-with-python).

In [2]:
%%bash
wget -q https://github.com/mnassrib/Titanic-logistic-regression-with-python/raw/master/logistic_regression_python.ipynb
mkdir data
wget -q https://github.com/mnassrib/Titanic-logistic-regression-with-python/raw/master/data/test.csv -O data/test.csv
wget -q https://github.com/mnassrib/Titanic-logistic-regression-with-python/raw/master/data/train.csv -O data/train.csv

Before using soorgeon to refactor the notebook into ploomber pipelines, we make some minor changes to the notebook:
* Ensure the original notebook is running. This is usually notebook-specific.
> `sklearn.feature_selection.RFE()` needs explicit passing of function argument `n_features_to_select`
 * We replace `RFE(model, 8)` to `RFE(model, n_features_to_select=8)`

* Soorgeon treats every H2 heading as starting point of a new task. We replace H1 headings to H2 headings.
> We replace sections 1 and 2 to H2 headings (e.g. from `"# 1."` to `"## 1."`).
---



In [3]:
# Minor changes to the notebook:
# * Only H2 headings are supported
# * `RFE` needs explicit passing of function argument `n_features_to_select`
replacements = {'"# 1.':'"## 1.', '"# 2.':'"## 2.', 'RFE(model, 8)': 'RFE(model, n_features_to_select=8)'}

with open('logistic_regression_python.ipynb') as infile, open('nb.ipynb', 'w') as outfile:
    for line in infile:
        for src, target in replacements.items():
            line = line.replace(src, target)
        outfile.write(line)


## Convert monolithic Jupyter notebooks 📙 into maintainable Ploomber pipelines with Soorgeon

### `Soorgeon refactor` will refactor the notebook into pipeline tasks in the `./tasks` folder.

In [4]:
%%bash
soorgeon refactor nb.ipynb

Added 'output' directory to .gitignore...
Added README.md
Finished refactoring 'nb.ipynb', use Ploomber to continue.

Install dependencies (this will install ploomber):
    $ pip install -r requirements.txt

List tasks:
    $ ploomber status

Execute pipeline:
    $ ploomber build

Plot pipeline:
    $ ploomber plot

* Documentation: https://docs.ploomber.io
* Jupyter integration: https://ploomber.io/s/jupyter
* Other editors: https://ploomber.io/s/editors



/usr/local/lib/python3.7/dist-packages/soorgeon/pyflakes.py:101: UserWarning: :4:1 'sklearn.preprocessing' imported but unused
:160:1 'sklearn.metrics.classification_report' imported but unused
:160:1 'sklearn.metrics.precision_score' imported but unused
:160:1 'sklearn.metrics.recall_score' imported but unused
:161:1 'sklearn.metrics.confusion_matrix' imported but unused
:161:1 'sklearn.metrics.precision_recall_curve' imported but unused
  warnings.warn(warnings_)


### Check out what is inside the `./tasks` folder

In [5]:
%%bash
ls tasks

section-1-import-data-python-packages.ipynb
section-2-1-age-missing-values.ipynb
section-2-2-cabin-missing-values.ipynb
section-2-3-embarked-missing-values.ipynb
section-2-4-1-additional-variables.ipynb
section-2-4-final-adjustments-to-data-train-test-.ipynb
section-2-data-quality-missing-value-assessment.ipynb
section-3-1-exploration-of-age.ipynb
section-3-2-exploration-of-fare.ipynb
section-3-3-exploration-of-passenger-class.ipynb
section-3-4-exploration-of-embarked-port.ipynb
section-3-5-exploration-of-traveling-alone-vs-with-family.ipynb
section-3-6-exploration-of-gender-variable.ipynb
section-4-1-feature-selection.ipynb
section-4-2-review-of-model-evaluation-procedures.ipynb
section-4-3-gridsearchcv-evaluating-using-multiple-scorers-simultaneously.ipynb
section-4-4-gridsearchcv-evaluating-using-multiple-scorers-repeatedstratifiedkfold-and-pipeline-for-preprocessing-simultaneously.ipynb


# Exposing pipelines to Airflow with Soopervisor

```yaml
airflow-bash:
  backend: airflow
  exclude: [output]
  preset: bash
  repository: your-repository/name
```


### Configure target platform

* We use BashOperator as the target platform.

* To configure output directory, we edit `env.yaml` to update the root variable For this example, let’s use one that we already configured.
```yaml
sample: False
root: /content
```
* Edit the `cwd` argument in `BashOperator` (`airflow-bash/content.py`) so your DAG runs in a directory where it can import your project’s `pipeline.yaml` and source code. For this example, we will also do this for you.

```python
    BashOperator(
        bash_command=task['command'],
        task_id=task['name'],
        dag=dag,
        cwd="/content" # add this line
    )
```


In [6]:
!pip freeze > requirements.lock.txt
# use BashOperator as the target platform
!soopervisor add airflow-bash --backend airflow --preset bash
# configure output directory in `env.yaml`
!wget https://github.com/ploomber/projects/raw/master/guides/airflow/env-airflow.yaml -O env.yaml -q
!soopervisor export airflow-bash --skip-tests --ignore-git

# Edit the cwd argument in BashOperator
!wget https://github.com/ploomber/projects/raw/master/guides/airflow/content.py -O airflow-bash/content.py -q

================================= Loading DAG ==================================
No pipeline.airflow-bash.yaml found, looking for pipeline.yaml instead
Found /content/pipeline.yaml. Loading...
Exporting to Airflow...
=================== Adding /content/airflow-bash/bash.py... ====================
Airflow DAG declaration saved to 'airflow-bash/content.py', you may edit the file to change the configuration if needed, (e.g., set the execution period)
================================= Loading DAG ==================================
No pipeline.airflow-bash.yaml found, looking for pipeline.yaml instead
/usr/local/lib/python3.7/dist-packages/ploomber/spec/dagspec.py:393: UserWarning: The following placeholders are declared in the environment but unused in the spec: {'root', 'sample'}
  warnings.warn('The following placeholders are declared in the '
Found /content/pipeline.yaml. Loading...
================================= Loading DAG ==================================
No pipeline.airflow-bash

### Submitting pipeline

To execute the pipeline, move the generated files to your `AIRFLOW_HOME`. For this example, AIRFLOW_HOME is `~/airflow`

In [7]:
%%bash
mkdir -p ~/airflow/dags && cp airflow-bash/content.py ~/airflow/dags/content.py && cp airflow-bash/content.json ~/airflow/dags/content.json 

Initize Airflow webserver and scheduler

In [8]:
%%bash
# init webserver in the background
airflow webserver --port 8080 > /airflow-webserver.log 2>&1 &

# init scheduler in the background
airflow scheduler > /airflow-scheduler.log 2>&1 &

# sleep a bit to ensure the scheduler is ready
sleep 10

Unpause the DAG then trigger the run:

In [9]:
%%bash
airflow dags unpause content

Dag: content, paused: False


In [10]:
%%bash
# Trigger execution
airflow dags trigger content

[2022-08-05 22:13:56,328] {__init__.py:40} INFO - Loaded API auth backend: airflow.api.auth.backend.session
Created <DagRun content @ 2022-08-05T22:13:56+00:00: manual__2022-08-05T22:13:56+00:00, externally triggered: True>


## Monitoring execution status

Check all runs of one dag_id:

`airflow dags list-runs -d {dag-id}`

In [12]:
%%bash
airflow dags list-runs -d content

dag_id  | run_id                            | state   | execution_date            | start_date                       | end_date
========+===================================+=========+===========================+==================================+=========
content | manual__2022-08-05T22:13:56+00:00 | running | 2022-08-05T22:13:56+00:00 | 2022-08-05T22:13:57.755890+00:00 |         
                                                                                                                               


Running `airflow dags state {dag_id} {TIMESTAMP}` 

(Find the `TIMESTAMP` from the output of `airflow dags trigger content`)

In [13]:
%%bash
airflow dags state content 2022-08-05T22:13:56+00:00

running


## Where to go from here

**Bring your own code!** Check out the tutorial to [migrate your code to Ploomber](https://docs.ploomber.io/en/latest/user-guide/refactoring.html) and [to expose Ploomber pipelines to Airflow, AWS Batch and Kubernetes](https://soopervisor.readthedocs.io/en/latest/).

Have questions? [Ask us anything on Slack](https://ploomber.io/community/).

Want to dig deeper into Ploomber's core concepts? Check out [the basic concepts tutorial](https://docs.ploomber.io/en/latest/get-started/basic-concepts.html).

Want to start a new project quickly? Check out [how to get examples](https://docs.ploomber.io/en/latest/user-guide/templates.html).
